# Анализ целостности набора данных

на примере датасета о черепахах

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
D = pd.read_excel('Data_S1_Complete_Dataset.xlsx', skiprows=5)
D=D.iloc[:,6:-5]
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692 entries, 0 to 691
Data columns (total 16 columns):
Genus               692 non-null object
Species             692 non-null object
Common Name         306 non-null object
Family              692 non-null object
M Mass (g)          297 non-null float64
F Mass (g)          297 non-null float64
N                   446 non-null object
Locality            289 non-null object
M SCL (mm)          534 non-null float64
F SCL (mm)          537 non-null float64
M Max Mass (g)      126 non-null float64
F Max Mass (g)      127 non-null float64
Pop Max Mass (g)    113 non-null float64
M Max SCL (mm)      271 non-null float64
F Max SCL (mm)      272 non-null float64
Pop Max SCL (mm)    125 non-null float64
dtypes: float64(10), object(6)
memory usage: 70.3+ KB


Для анализа полового диморфизма каждая запись должна содержать размер женской особи и мужской особи.

Вопрос: сколько размеров без пары?

Первая проверка, сколько мужских без женских?

In [3]:
D[ D['M SCL (mm)'].notnull() & D['F SCL (mm)'].isnull() ]

,Genus,Species,Common Name,Family,M Mass (g),F Mass (g),N,Locality,M SCL (mm),F SCL (mm),M Max Mass (g),F Max Mass (g),Pop Max Mass (g),M Max SCL (mm),F Max SCL (mm),Pop Max SCL (mm)


Вторая проверка, сколько женских без мужских?

In [4]:
D[ D['M SCL (mm)'].isnull() & D['F SCL (mm)'].notnull() ]

,Genus,Species,Common Name,Family,M Mass (g),F Mass (g),N,Locality,M SCL (mm),F SCL (mm),M Max Mass (g),F Max Mass (g),Pop Max Mass (g),M Max SCL (mm),F Max SCL (mm),Pop Max SCL (mm)
156,Cycloderma,frenatum,Zambezi flapshell turtle,Trionychidae,NaN,NaN,"1M, 1F","Zambezi River, southern Africa",NaN,560.0,13500.0,11400.0,NaN,NaN,NaN,NaN
466,Pelomedusa,subrufa,helmeted turtle,Pelomedusidae,NaN,NaN,NaN,Sub-Saharan Africa,NaN,290.0,3000.0,2200.0,3000.0,325.0,NaN,NaN
471,Pelusios,adansonii,Adanson's turtle,Pelomedusidae,NaN,NaN,NaN,Senegal,NaN,220.0,650.0,1620.0,NaN,NaN,238.0,NaN


In [5]:
_.shape

(3, 16)

В трех случаях указаны только женские размеры.

Для Pelomedusa 	subrufa (№ 466) правда указан максимальный размер самца.

Можно заметить, что все три вида - морские черепахи. Поэтому становится понятным почему нет самцов.

Вероятно, замеры самок производились во время кладки яиц, когда самки черепах выползают из воды на песок. Самцы же в это время плавают где-то в море.

### Анализ корректности семейств

Для сравнительного анализа часто необходимо группировать наблюдения. В этом наборе данных группировку логично делать по систематическому признаку - Семейству (Family).

Проверим, что у всех видов указано семейство.

In [2]:
D.Family.value_counts().sort_index()

NameError: name 'D' is not defined

Обратите внимание, что Chelydridae встречаются два раза. Это означает, что эти названия различаются на один невидимый символ. 

In [7]:
D.Family.unique()

array([u'Chelidae', u'Emydidae', u'Testudinidae', u'Trionychidae',
       u'Geoemydidae', u'Cheloniidae', u'Carettochelyidae',
       u'Chelydridae ', u'Chelydridae', u'Kinosternidae',
       u'Dermatemydidae', u'Dermochelyidae', u'Podocnemididae',
       u'Pelomedusidae', u'Platysternidae'], dtype=object)

Отбираем все с ошибочным написанием и заменяем значение в колонке Family на правильное.

In [8]:
D.ix[D.Family=='Chelydridae ','Family']='Chelydridae'

D.Family.unique()

array([u'Chelidae', u'Emydidae', u'Testudinidae', u'Trionychidae',
       u'Geoemydidae', u'Cheloniidae', u'Carettochelyidae', u'Chelydridae',
       u'Kinosternidae', u'Dermatemydidae', u'Dermochelyidae',
       u'Podocnemididae', u'Pelomedusidae', u'Platysternidae'], dtype=object)

Если посмотреть на ту же проблему с названиями родов, то окажется, что это типичная ситуация.

In [9]:
sum(D.Genus.str.endswith(' '))  #проверяем, что строка с названием рода оканчивается пробелом

131

Примерно каждое пятое название рода оканчивается пробелом. Т.е. авторы не придавали этому значения. Когда строки содержат пробелы спереди или сзади как правило это не влияет на их смысловое содержание. Для обрезания крайних пробелов существует специальная функция `strip`, после применения которой дублей не остается.

In [10]:
unique(D.Genus.str.strip())

array([u'Acanthochelys', u'Agrionemys', u'Apalone', u'Aspideretes',
       u'Astrochelys', u'Batagur', u'Caretta', u'Carettochelys',
       u'Chelodina', u'Chelonia', u'Chelonoidis', u'Chelus', u'Chelydra',
       u'Chersina', u'Chitra', u'Chrysemys', u'Claudius', u'Clemmys',
       u'Cuora', u'Cyclemys', u'Cycloderma', u'Deirochelys', u'Dermatemys',
       u'Dermochelys', u'Dipsochelys', u'Dogania', u'Elseya', u'Elusor',
       u'Emydura', u'Emys', u'Eretmochelys', u'Erymnochelys',
       u'Geochelone', u'Geoclemys', u'Glyptemys', u'Gopherus',
       u'Graptemys', u'Hardella', u'Heosemys', u'Homopus', u'Hydromedusa',
       u'Indotestudo', u'Kinixys', u'Kinosternon', u'Kinosternum',
       u'Lepidochelys', u'Leucocephalon', u'Lissemys', u'Macrochelys',
       u'Malaclemys', u'Malacochersus', u'Malayemys', u'Manouria',
       u'Mauremys', u'Melanochelys', u'Mesoclemmys', u'Morenia',
       u'Myuchelys', u'Nilssonia', u'Notochelys', u'Palea', u'Pangshura',
       u'Pelodiscus', u'Pelome

В подобном случае операцию приведения строк в порядок можно делать сразу после загрузки набора данных. Например, мы все значения из колонки меняем на эти же значения, но исправленные.
```py
D.Genus = D.Genus.str.strip()
```
Похожие операция стандартизации могут потребовать нестандартные сокращения, нестандартные указания даты и времени, строки с разным регистром символов и др.

После "ремонта" набора данных его желательно сохранить в файле с другим именем для дальнейшей работы. Мы этого делать не будем, поскольку преследуем учебные цели.

In [ ]:
D.Genus + ' ' +D.Species